In [1]:
#pip install pyspark

In [2]:
#conda install -c cyclus java-jdk

In [3]:
#pip install py4j

In [4]:
#load pySpark modules 
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql import functions as F
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
from pyspark.sql.types import *
from pyspark.sql.functions import *
import time
spark = SparkSession.builder.getOrCreate()
spark = SparkSession.builder \
    .master('local[*]') \
    .config("spark.driver.memory", "30g") \
    .config('spark.executor.memory=30g')\
    .appName('final') \
    .getOrCreate()
sc=spark.sparkContext

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/06 12:27:18 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/05/06 12:27:21 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/05/06 12:27:21 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/05/06 12:27:21 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/05/06 12:27:21 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.


In [5]:
#.config("spark.driver.memory", "15g") \

In [6]:
#load flight data file into pySpark dataFrame format

flight = spark.read.csv("flights.csv", inferSchema=True, header=True)
airports = spark.read.csv("airports.csv", inferSchema=True, header=True)
flight.show(5)
airports.show(5)


+----------+---------+-------+---------------+-------------+--------+--------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|ArrDelay|
+----------+---------+-------+---------------+-------------+--------+--------+
|        19|        5|     DL|          11433|        13303|      -3|       1|
|        19|        5|     DL|          14869|        12478|       0|      -8|
|        19|        5|     DL|          14057|        14869|      -4|     -15|
|        19|        5|     DL|          15016|        11433|      28|      24|
|        19|        5|     DL|          11193|        12892|      -6|     -11|
+----------+---------+-------+---------------+-------------+--------+--------+
only showing top 5 rows

+----------+-----------+-----+--------------------+
|airport_id|       city|state|                name|
+----------+-----------+-----+--------------------+
|     10165|Adak Island|   AK|                Adak|
|     10299|  Anchorage|   AK|Ted Stevens Ancho...|
|   

In [7]:
df_dis = flight.distinct()
df_f = df_dis.select('DayofMonth','DayOfWeek','Carrier','OriginAirportID','DestAirportID','DepDelay',((col("ArrDelay") > 15).cast("Int").alias("label")))
df_f = df_f.withColumnRenamed("label", "trueLabel")
df_f.show(5)

+----------+---------+-------+---------------+-------------+--------+---------+
|DayofMonth|DayOfWeek|Carrier|OriginAirportID|DestAirportID|DepDelay|trueLabel|
+----------+---------+-------+---------------+-------------+--------+---------+
|         6|        1|     WN|          10821|        10140|       1|        0|
|         8|        1|     AA|          11298|        10140|       0|        0|
|        15|        1|     WN|          14747|        10140|      -6|        0|
|        27|        1|     AA|          11298|        10140|     113|        1|
|         7|        2|     OO|          12266|        10140|      -3|        0|
+----------+---------+-------+---------------+-------------+--------+---------+
only showing top 5 rows



### Define the Pipeline and split the data 

In [24]:
from pyspark.ml.feature import VectorAssembler, StringIndexer, VectorIndexer, MinMaxScaler
strIdx = StringIndexer(inputCol = "Carrier", outputCol = "CarrierIdx")
catVect = VectorAssembler(inputCols = ["CarrierIdx", "DayofMonth", "DayOfWeek", "OriginAirportID", "DestAirportID"], outputCol="catFeatures")
catIdx = VectorIndexer(inputCol = catVect.getOutputCol(), outputCol = "idxCatFeatures")
numVect = VectorAssembler(inputCols = ["DepDelay"], outputCol="numFeatures")
minMax = MinMaxScaler(inputCol = numVect.getOutputCol(), outputCol="normFeatures")
featVect = VectorAssembler(inputCols=["idxCatFeatures", "normFeatures"], outputCol="features")
label_stringIdx = StringIndexer(inputCol = 'trueLabel', outputCol = 'label')
train, test = df_f.randomSplit([0.8, 0.2], seed = 2018)


### build the LinearSVC model

In [25]:
from pyspark.ml.classification import LinearSVC
from pyspark.ml import Pipeline
# Train a DecisionTree model.
SVC = LinearSVC(labelCol="label",featuresCol="features",maxIter=10, regParam=0.1)
pipeline = Pipeline(stages=[strIdx, catVect, catIdx, numVect, minMax, featVect,label_stringIdx, SVC])
# Fit the pipeline to training documents.
t1 = time.time()
SVCmodel = pipeline.fit(train)
t2 = time.time()
print(t2-t1)

22/05/06 17:56:31 WARN MemoryStore: Not enough space to cache rdd_1732_1 in memory! (computed 17.0 MiB so far)
22/05/06 17:56:31 WARN BlockManager: Persisting block rdd_1732_1 to disk instead.


70.66387987136841


#### Make predictions on the test set

In [26]:
predictions = SVCmodel.transform(test)
predictions_SVC = predictions.select("features", 'label',"prediction", "trueLabel")
predictions_SVC.show(5)
#predictions.show(5)

+--------------------+-----+----------+---------+
|            features|label|prediction|trueLabel|
+--------------------+-----+----------+---------+
|[10.0,1.0,0.0,103...|  0.0|       0.0|        0|
|[10.0,1.0,0.0,104...|  0.0|       0.0|        0|
|[10.0,1.0,0.0,107...|  0.0|       0.0|        0|
|[10.0,1.0,0.0,107...|  1.0|       0.0|        1|
|[10.0,1.0,0.0,107...|  0.0|       0.0|        0|
+--------------------+-----+----------+---------+
only showing top 5 rows



#### Evaluate our  model.

In [27]:
#Evaluating prediction results

tp = float(predictions.filter("prediction == 1.0 AND label == 1").count())
fp = float(predictions.filter("prediction == 1.0 AND label == 0").count())
tn = float(predictions.filter("prediction == 0.0 AND label == 0").count())
fn = float(predictions.filter("prediction == 0.0 AND label == 1").count())
pr = tp / (tp + fp)
re = tp / (tp + fn)
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", pr),
 ("Recall", re),
 ("F1", 2*pr*re/(re+pr))],["metric", "value"])
metrics.show()

+---------+------------------+
|   metric|             value|
+---------+------------------+
|       TP|           45639.0|
|       FP|             235.0|
|       TN|          431220.0|
|       FN|           61751.0|
|Precision|0.9948772725291014|
|   Recall|0.4249837042555173|
|       F1|0.5955606013153775|
+---------+------------------+



In [28]:
#Evaluate our Logistic Regression model.
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = BinaryClassificationEvaluator()
print('Test Area Under ROC', evaluator.evaluate(predictions))

Test Area Under ROC 0.9236887751845569


In [29]:
from sklearn import metrics
actual =predictions.select('label').toPandas()
predicted = predictions.select('prediction').toPandas()
accuracy = metrics.accuracy_score(actual, predicted)
print('accuracy = ', accuracy)

accuracy =  0.8849650641650196


In [30]:
train5, test5 = df_f.randomSplit([0.5, 0.5], seed = 2000)
train6, test6 = df_f.randomSplit([0.5, 0.5], seed = 2001)
train7, test7 = df_f.randomSplit([0.5, 0.5], seed = 2002)
train9, test9 = df_f.randomSplit([0.5, 0.5], seed = 2003)

In [31]:
#train the model
t1 = time.time()
SVCmodel = pipeline.fit(train5)
t2 = time.time()
print(t2-t1)
#predict the model
predictions5 = SVCmodel.transform(test5)
#evaluate the model
tp = float(predictions5.filter("prediction == 1.0 AND label == 1").count())
fp = float(predictions5.filter("prediction == 1.0 AND label == 0").count())
tn = float(predictions5.filter("prediction == 0.0 AND label == 0").count())
fn = float(predictions5.filter("prediction == 0.0 AND label == 1").count())
pr = tp / (tp + fp)
re = tp / (tp + fn)
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", pr),
 ("Recall", re),
 ("F1", 2*pr*re/(re+pr))],["metric", "value"])
metrics.show()
#evaluate AUC
evaluator = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
print('AUC = ', evaluator.evaluate(predictions5))


22/05/06 17:58:48 WARN MemoryStore: Not enough space to cache rdd_1938_1 in memory! (computed 17.0 MiB so far)
22/05/06 17:58:48 WARN BlockManager: Persisting block rdd_1938_1 to disk instead.


58.84494376182556


+---------+-------------------+
|   metric|              value|
+---------+-------------------+
|       TP|           113687.0|
|       FP|              632.0|
|       TN|          1078855.0|
|       FN|           155071.0|
|Precision| 0.9944716101435457|
|   Recall|0.42300880345887376|
|       F1| 0.5935464671593439|
+---------+-------------------+



AUC =  0.924463975353851


In [32]:
#train the model
t1 = time.time()
SVCmodel = pipeline.fit(train6)
t2 = time.time()
print(t2-t1)
#predict the model
predictions6 = SVCmodel.transform(test6)
#evaluate the model
tp = float(predictions6.filter("prediction == 1.0 AND label == 1").count())
fp = float(predictions6.filter("prediction == 1.0 AND label == 0").count())
tn = float(predictions6.filter("prediction == 0.0 AND label == 0").count())
fn = float(predictions6.filter("prediction == 0.0 AND label == 1").count())
pr = tp / (tp + fp)
re = tp / (tp + fn)
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", pr),
 ("Recall", re),
 ("F1", 2*pr*re/(re+pr))],["metric", "value"])
metrics.show()
#evaluate AUC
evaluator = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
print('AUC = ', evaluator.evaluate(predictions6))


62.54458689689636


+---------+------------------+
|   metric|             value|
+---------+------------------+
|       TP|          113465.0|
|       FP|             612.0|
|       TN|         1078260.0|
|       FN|          155985.0|
|Precision|0.9946352025386361|
|   Recall|0.4210985340508443|
|       F1|0.5916923710716585|
+---------+------------------+



AUC =  0.9244869283681898


In [33]:
#train the model
t1 = time.time()
SVCmodel = pipeline.fit(train7)
t2 = time.time()
print(t2-t1)
#predict the model
predictions7 = SVCmodel.transform(test7)
#evaluate the model
tp = float(predictions7.filter("prediction == 1.0 AND label == 1").count())
fp = float(predictions7.filter("prediction == 1.0 AND label == 0").count())
tn = float(predictions7.filter("prediction == 0.0 AND label == 0").count())
fn = float(predictions7.filter("prediction == 0.0 AND label == 1").count())
pr = tp / (tp + fp)
re = tp / (tp + fn)
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", pr),
 ("Recall", re),
 ("F1", 2*pr*re/(re+pr))],["metric", "value"])
metrics.show()
#evaluate AUC
evaluator = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
print('AUC = ', evaluator.evaluate(predictions7))

22/05/06 18:02:53 WARN MemoryStore: Not enough space to cache rdd_2308_1 in memory! (computed 17.0 MiB so far)
22/05/06 18:02:53 WARN BlockManager: Persisting block rdd_2308_1 to disk instead.


64.90484881401062


+---------+-------------------+
|   metric|              value|
+---------+-------------------+
|       TP|           113704.0|
|       FP|              590.0|
|       TN|          1078911.0|
|       FN|           154949.0|
|Precision| 0.9948378742541166|
|   Recall|0.42323741033973195|
|       F1| 0.5938367450326025|
+---------+-------------------+



AUC =  0.9247578464423215


In [34]:
#train the model
t1 = time.time()
SVCmodel = pipeline.fit(train9)
t2 = time.time()
print(t2-t1)
#predict the model
predictions9 = SVCmodel.transform(test9)
#evaluate the model
tp = float(predictions9.filter("prediction == 1.0 AND label == 1").count())
fp = float(predictions9.filter("prediction == 1.0 AND label == 0").count())
tn = float(predictions9.filter("prediction == 0.0 AND label == 0").count())
fn = float(predictions9.filter("prediction == 0.0 AND label == 1").count())
pr = tp / (tp + fp)
re = tp / (tp + fn)
metrics = spark.createDataFrame([
 ("TP", tp),
 ("FP", fp),
 ("TN", tn),
 ("FN", fn),
 ("Precision", pr),
 ("Recall", re),
 ("F1", 2*pr*re/(re+pr))],["metric", "value"])
metrics.show()
#evaluate AUC
evaluator = BinaryClassificationEvaluator(labelCol="trueLabel", rawPredictionCol="rawPrediction", metricName="areaUnderROC")
print('AUC = ', evaluator.evaluate(predictions9))

22/05/06 18:05:17 WARN MemoryStore: Not enough space to cache rdd_2493_3 in memory! (computed 17.0 MiB so far)
22/05/06 18:05:17 WARN BlockManager: Persisting block rdd_2493_3 to disk instead.


57.11459517478943


+---------+-------------------+
|   metric|              value|
+---------+-------------------+
|       TP|           113778.0|
|       FP|              602.0|
|       TN|          1079021.0|
|       FN|           154786.0|
|Precision| 0.9947368421052631|
|   Recall|0.42365320742914164|
|       F1| 0.5942278766608172|
+---------+-------------------+



AUC =  0.9248464750926111


In [35]:
from sklearn import metrics
actual5 =predictions5.select('label').toPandas()
predicted5 = predictions5.select('prediction').toPandas()
accuracy5 = metrics.accuracy_score(actual5, predicted5)
print('accuracy = ', accuracy5)

accuracy =  0.8845143130514113


In [36]:
#evaluate accuracy
actual6 =predictions6.select('label').toPandas()
predicted6 = predictions6.select('prediction').toPandas()
accuracy6= metrics.accuracy_score(actual6, predicted6)
print('accuracy6 = ', accuracy6)

accuracy6 =  0.8838578618460575


In [37]:
#evaluate accuracy
actual7 =predictions7.select('label').toPandas()
predicted7 = predictions7.select('prediction').toPandas()
accuracy7 = metrics.accuracy_score(actual7, predicted7)
print('accuracy7 = ', accuracy7)

accuracy7 =  0.8846281656249954


In [38]:
#evaluate accuracy
actual9 =predictions9.select('label').toPandas()
predicted9 = predictions9.select('prediction').toPandas()
accuracy9 = metrics.accuracy_score(actual9, predicted9)
print('accuracy9 = ', accuracy9)

accuracy9 =  0.884742991884657


In [ ]:
#sc.stop()